In [1]:
!pip install ipywidgets

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torch import optim
from torchvision import datasets
from torchvision.transforms import ToTensor, Lambda, Compose, functional, RandomResizedCrop, Resize


In [3]:
# Training data
data_transform = Compose([
                                        RandomResizedCrop(500),
                                        ToTensor(),
                                        Resize(256),
                                        ])

training_data = datasets.Flowers102(
                        root="data",
                        split="train",
                        download=True,
                        transform=data_transform,
                        )

In [4]:
test_data = datasets.Flowers102(
                        root="data",
                        split="test",
                        download=True,
                        transform=data_transform,
                        )


In [5]:
training_data[1][0].size()

torch.Size([3, 256, 256])

In [6]:
batch_sz = 64
train_dataloader = DataLoader(training_data, batch_size=batch_sz)
test_dataloader = DataLoader(test_data, batch_size=batch_sz)


In [7]:
for X, y in test_dataloader:
    print(f"Shape of X [N, C, H, W]: {X.shape}")
    print(f"Shape of y: {y.shape} {y.dtype}")
    break
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using {device} device")


Shape of X [N, C, H, W]: torch.Size([64, 3, 256, 256])
Shape of y: torch.Size([64]) torch.int64
Using cuda device


In [8]:
class FlowerNet(nn.Module):
    def __init__(self):
        super(FlowerNet, self).__init__()
        self.flatten = nn.Flatten()
        self.conv_stack = nn.Sequential(
            nn.Conv2d(3, 20, 5, stride=1, dilation = 0 ),
            nn.ReLU(),
            nn.MaxPool2d(5,stride=1,dilation=0),
            nn.Conv2d(20, 50, 5, stride=1, dilation = 0),
            nn.ReLU(),
            nn.MaxPool2d(5,stride=1,dilation=0)
        )
        
        self.linear_stack = nn.Sequential(
            nn.Linear(4050, 512),
            nn.ReLU(),
            nn.Linear(512, 102),
            nn.ReLU()
        )

    def forward(self, x):
        logits = self.conv_stack(x)
        logits = torch.flatten(logits,1,-1)
        # print(f"flatted output{logits.size}")
        logits = self.linear_stack(logits)

        return logits

model = FlowerNet().to(device)
print(model)

FlowerNet(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (conv_stack): Sequential(
    (0): Conv2d(3, 20, kernel_size=(5, 5), stride=(1, 1), dilation=(0, 0))
    (1): ReLU()
    (2): MaxPool2d(kernel_size=5, stride=1, padding=0, dilation=0, ceil_mode=False)
    (3): Conv2d(20, 50, kernel_size=(5, 5), stride=(1, 1), dilation=(0, 0))
    (4): ReLU()
    (5): MaxPool2d(kernel_size=5, stride=1, padding=0, dilation=0, ceil_mode=False)
  )
  (linear_stack): Sequential(
    (0): Linear(in_features=4050, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=102, bias=True)
    (3): ReLU()
  )
)


In [9]:
# Define optimizer
learning_rate = 0.0001
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
loss_criterion = nn.CrossEntropyLoss()
num_epochs = 50
# in your training loop:
for i in range(num_epochs):
  print(f"Running Epoch number: {i}")
  batch = 0;
  for x,y in train_dataloader:
    if(y.shape[0] != batch_sz):
      continue
    optimizer.zero_grad()   # zero the gradient buffers
    # print(x.shape, x.dtype)
    x = x.to(device)
    y = y.to(device)
    output = model(x)
    # print(output.shape)
    # print(y.shape)
    loss = loss_criterion(output, y)
    loss.backward()
    optimizer.step()    # Does the updatefrom torch.utils.data import DataLoader
    batch+=1
    print(f"Loss value is: {loss}")


Running Epoch number: 0


RuntimeError: ignored